# Tensorflow solution to Project 1

## Importing modules and preprocessing

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

In [2]:
data_path = 'Bike-Sharing-Dataset/hour.csv'

rides = pd.read_csv(data_path)

In [3]:
rides.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)
data.head()

,yr,holiday,temp,hum,windspeed,casual,registered,cnt,season_1,season_2,...,hr_21,hr_22,hr_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0.24,0.81,0.0,3,13,16,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0.22,0.80,0.0,8,32,40,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0.22,0.80,0.0,5,27,32,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0.24,0.75,0.0,3,10,13,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0.24,0.75,0.0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

In [6]:
# Save data for approximately the last 21 days 
test_data = data[-21*24:]

# Now remove the test data from the data set 
data = data[:-21*24]

# Separate the data into features and targets
target_fields = ['cnt', 'casual', 'registered']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [7]:
# Hold out the last 60 days or so of the remaining data as a validation set
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

## Building a single layer neural network with Tensorflow

In [ ]:
train_targets.shape

In [ ]:
val_features.shape

In [ ]:
val_targets.head()

In [ ]:
def neural_net(input, num_features):
    # Build input layer
    labels_count = 1

    num_hidden = 8

    weights_i_h = tf.Variable(tf.truncated_normal((num_features, num_hidden)))
    biases_i_h = tf.Variable(tf.zeros(num_hidden))

    weights_h_o = tf.Variable(tf.truncated_normal((num_hidden, labels_count)))
    biases_h_o = tf.Variable(tf.zeros(labels_count))
    
    logits_i_h = tf.matmul(inputs, weights_i_h) + biases_i_h # when to use tf.add?
    sigmoid_i_h = tf.nn.sigmoid(logits_i_h, name=None)

    prediction = tf.matmul(sigmoid_i_h, weights_h_o) + biases_h_o
    return prediction

In [ ]:
def train_neural_network(x, num_features):
    prediction = neural_net(x, num_features) #feed-forward
    loss = tf.reduce_mean(tf.squared_difference(prediction, label))
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)    # default learning_rate = 0.001

    # cycles of feedforwards + backprop
    hm_epochs = 10000
    training_fd = {inputs: train_features, label: train_targets["cnt"][:,None]}
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):      
            _, c = sess.run([optimizer, loss], feed_dict = training_fd)
            if (epoch % 1000 == 0 or epoch == 9999):
                print("Epoch", epoch, "completed out of", hm_epochs, "loss:", c)



In [ ]:
num_features = train_features.shape[1]
labels_count = 1
inputs = tf.placeholder(tf.float32, [None, num_features])
label = tf.placeholder(tf.float32, [None, labels_count])
train_neural_network(inputs, num_features)

## Adding more layers

In [ ]:
def two_hidden_layer_nn(input, num_features):
    # Build input layer
    labels_count = 1

    # layer 1
    num_hidden_1 = 8
    weights_i_h1 = tf.Variable(tf.truncated_normal((num_features, num_hidden_1)))
    biases_i_h1 = tf.Variable(tf.zeros(num_hidden_1))
    logits_i_h1 = tf.matmul(inputs, weights_i_h1) + biases_i_h1 # when to use tf.add?
    sigmoid_i_h1 = tf.nn.sigmoid(logits_i_h1, name=None)
    
    #layer 2
    num_hidden_2 = 8
    weights_h1_h2 = tf.Variable(tf.truncated_normal((num_hidden_1, num_hidden_2)))
    biases_h1_h2 = tf.Variable(tf.zeros(num_hidden_2))
    logits_h1_h2 = tf.matmul(sigmoid_i_h1, weights_h1_h2) + biases_h1_h2 # when to use tf.add?
    sigmoid_h1_h2 = tf.nn.sigmoid(logits_h1_h2, name=None)
    
    #output
    weights_h2_o = tf.Variable(tf.truncated_normal((num_hidden_2, labels_count)))
    biases_h2_o = tf.Variable(tf.zeros(labels_count))
    prediction = tf.matmul(sigmoid_h1_h2, weights_h2_o) + biases_h2_o
    
    return prediction

In [ ]:
def train_two_hidden_nn(x, num_features):
    prediction = two_hidden_layer_nn(x, num_features) #feed-forward
    loss = tf.reduce_mean(tf.squared_difference(prediction, label))
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)    # default learning_rate = 0.001

    # cycles of feedforwards + backprop
    hm_epochs = 10000
    training_fd = {inputs: train_features, label: train_targets["cnt"][:,None]}
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in tqdm(range(hm_epochs)):      
            _, c = sess.run([optimizer, loss], feed_dict = training_fd)
            if (epoch % 1000 == 0 or epoch == 9999):
                print("Epoch", epoch, "completed out of", hm_epochs, "loss:", c)

In [ ]:
train_two_hidden_nn(inputs, num_features)

## Refactor code to allow for saving and loading models

### First, build the computational graph. The cell below is to be copy-pasted when loading a pre-trained model

In [24]:
# Building the computational graph
num_features = train_features.shape[1]
labels_count = 1

inputs = tf.placeholder(tf.float32, [None, num_features])
label = tf.placeholder(tf.float32, [None, labels_count])


num_hidden_1 = 8
num_hidden_2 = 8

weights_i_h1 = tf.Variable(tf.truncated_normal((num_features, num_hidden_1)))
biases_i_h1 = tf.Variable(tf.zeros(num_hidden_1))
weights_h1_h2 = tf.Variable(tf.truncated_normal((num_hidden_1, num_hidden_2)))
biases_h1_h2 = tf.Variable(tf.zeros(num_hidden_2))
weights_h2_o = tf.Variable(tf.truncated_normal((num_hidden_2, labels_count)))
biases_h2_o = tf.Variable(tf.zeros(labels_count))

logits_i_h1 = tf.matmul(inputs, weights_i_h1) + biases_i_h1 # when to use tf.add?
sigmoid_i_h1 = tf.nn.sigmoid(logits_i_h1, name=None)

logits_h1_h2 = tf.matmul(sigmoid_i_h1, weights_h1_h2) + biases_h1_h2 # when to use tf.add?
sigmoid_h1_h2 = tf.nn.sigmoid(logits_h1_h2, name=None)

prediction = tf.matmul(sigmoid_h1_h2, weights_h2_o) + biases_h2_o
loss = tf.reduce_mean(tf.squared_difference(prediction, label))
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)    # default learning_rate = 0.001


### Next train the model, construct a dictionary containing all the model parameters (numpy) as values and the filenames as keys. Save all variables in the dictionary as numpy files

In [19]:
# Training the model
# cycles of feedforwards + backprop
hm_epochs = 10000
training_fd = {inputs: train_features, label: train_targets["cnt"][:,None]}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in tqdm(range(hm_epochs)):      
        _, c = sess.run([optimizer, loss], feed_dict = training_fd)
        if (epoch % 1000 == 0 or epoch == 9999):
            print("Epoch", epoch, "completed out of", hm_epochs, "loss:", c)
    print("saving variables...")
    param_dict = {"wih1.npy": weights_i_h1, "bih1": biases_i_h1, "wh1h2": weights_h1_h2, 
                  "bh1h2": biases_h1_h2, "wh20": weights_h2_o, "bh20": biases_h2_o}
    for file, param in param_dict.items():
        np.save(file, param.eval())
    print("Done!")
    

  0%|          | 9/10000 [00:00<05:20, 31.15it/s]

Epoch 0 completed out of 10000 loss: 1.28179


 10%|█         | 1007/10000 [00:23<03:21, 44.58it/s]

Epoch 1000 completed out of 10000 loss: 0.345041


 20%|██        | 2009/10000 [00:46<02:40, 49.69it/s]

...

Epoch 8000 completed out of 10000 loss: 0.097107


 90%|█████████ | 9007/10000 [03:10<00:20, 48.78it/s]

Epoch 9000 completed out of 10000 loss: 0.0880489


100%|██████████| 10000/10000 [03:31<00:00, 49.25it/s]

Epoch 9999 completed out of 10000 loss: 0.0818711
saving variables...
Done!


### The cell below can be run by itself. First, copy and paste the graph structure. This time, instead of randomly initialising values, load the numpy arrays back into the model parameter variables. Finally, run the graph as per normal with the test data.

In [23]:
import load

num_features = train_features.shape[1]
labels_count = 1

inputs = tf.placeholder(tf.float32, [None, num_features])
label = tf.placeholder(tf.float32, [None, labels_count])

params_dict = load.run()
weights_i_h1 = tf.Variable(params_dict["wih1"])
biases_i_h1 = tf.Variable(params_dict["bih1"])
weights_h1_h2 = tf.Variable(params_dict["wh1h2"])
biases_h1_h2 = tf.Variable(params_dict["bh1h2"])
weights_h2_o = tf.Variable(params_dict["wh20"])
biases_h2_o = tf.Variable(params_dict["bh20"])

logits_i_h1 = tf.matmul(inputs, weights_i_h1) + biases_i_h1 # when to use tf.add?
sigmoid_i_h1 = tf.nn.sigmoid(logits_i_h1, name=None)

logits_h1_h2 = tf.matmul(sigmoid_i_h1, weights_h1_h2) + biases_h1_h2 # when to use tf.add?
sigmoid_h1_h2 = tf.nn.sigmoid(logits_h1_h2, name=None)

prediction = tf.matmul(sigmoid_h1_h2, weights_h2_o) + biases_h2_o
loss = tf.reduce_mean(tf.squared_difference(prediction, label))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    testing_fd = {inputs: test_features, label: test_targets["cnt"][:,None]}
    print("running prediction...")
    print(sess.run(loss, feed_dict=testing_fd))

running prediction...
0.228952
